In [108]:
%matplotlib inline
import numpy as np
import pandas as pd
import keras 
import os
import random
import cv2
from random import shuffle
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [98]:
random.seed(123)
train_dir = 'imgs/train'
test_dir = 'imgs/test'
batch_size = 64
epochs = 10
num_classes = 10
img_size = 128
lr = 1e-3

In [99]:
def label_img(img_label):
    label = int(list(img_label)[1])
    #print(label)
    l = [0]*10
    l[label]=1
    return l

def create_train_data():
    training_data=[]
    for d in os.listdir(train_dir):
        l = d
        d = os.path.join(train_dir,d)
        for img in os.listdir(d):
            label = label_img(l)
            img_path = os.path.join(d,img)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img,(img_size,img_size))
            training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save("training_data.npy",training_data)
    return training_data

def create_testing_data():
    testing_data=[]
    for img in os.listdir(test_dir):
        img_path = os.path.join(test_dir,img)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img,(img_size,img_size))
        testing_data.append([np.array(img)])
    shuffle(testing_data)
    np.save("testing_data.npy",testing_data)
    return testing_data

In [100]:
if os.path.exists("training_data.npy"):
    train_data = np.load("training_data.npy")
else:
    train_data = create_train_data()

In [101]:
img = train_data[0][0]
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [102]:
len(train_data)

22424

In [103]:
train = train_data[:-500]
test = train_data[-500:]

In [104]:
X = np.array([i[0] for i in train]).reshape(-1,img_size,img_size,1)
y = np.array([i[1] for i in train])
test_x = np.array([i[0] for i in test]).reshape(-1,img_size,img_size,1)
test_y = np.array([i[1] for i in test])

In [111]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding = 'same',activation = 'relu',input_shape=(img_size,img_size,1),bias_initializer=keras.initializers.Ones()))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))

model.add(Conv2D(64,kernel_size=(3,3),padding = 'same',activation = 'relu',bias_initializer=keras.initializers.Ones()))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))

model.add(Conv2D(128,kernel_size=(3,3), padding='same', activation='relu',bias_initializer=keras.initializers.Ones()))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

model.add(Flatten())
model.add(Dense(128, activation='relu', bias_initializer=keras.initializers.Ones()))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

In [112]:
model.compile(optimizer = keras.optimizers.SGD(lr=lr), loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])

In [113]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 128, 128, 32)      320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 32, 32, 128)       73856     
__________

In [114]:
checkpoint = ModelCheckpoint('check01', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
driver_train_model = model.fit(X,y,batch_size=batch_size, epochs=epochs,verbose = 1, validation_data=(test_x,test_y), callbacks= callbacks_list)

Train on 21924 samples, validate on 500 samples
Epoch 1/10
21924/21924 [==============================] - 1031s 47ms/step - loss: 0.7815 - acc: 0.7857 - val_loss: 0.2910 - val_acc: 0.9460

Epoch 00001: val_acc improved from -inf to 0.94600, saving model to check01
Epoch 2/10
21924/21924 [==============================] - 1134s 52ms/step - loss: 0.1781 - acc: 0.9760 - val_loss: 0.1478 - val_acc: 0.9800

Epoch 00002: val_acc improved from 0.94600 to 0.98000, saving model to check01
Epoch 3/10
21924/21924 [==============================] - 1113s 51ms/step - loss: 0.1011 - acc: 0.9898 - val_loss: 0.1005 - val_acc: 0.9840

Epoch 00003: val_acc improved from 0.98000 to 0.98400, saving model to check01
Epoch 4/10
21924/21924 [==============================] - 1151s 53ms/step - loss: 0.0694 - acc: 0.9949 - val_loss: 0.0773 - val_acc: 0.9860

Epoch 00004: val_acc improved from 0.98400 to 0.98600, saving model to check01
Epoch 5/10
21924/21924 [==============================] - 1004s 46ms/step -